In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
def main(image, annotation):

  orig_image = cv2.imread(image)[annotation[1]: annotation[3], annotation[0]: annotation[2]]
  orig_image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB)

  plt.figure()
  plt.imshow(orig_image)
  
  boundaries = ([150, 0, 0], [255, 255, 255])
  circles = generate_circles(image, annotation)
  
  if(circles is None):
    print("Error")
    return
  
  circles = circles[0]
  circles = sorted(circles, key=lambda x: x[1])
 
  if(len(circles) == 0):
    print("Error")
    return
  
  for circle in circles:
    
    x_coor = circle[0]
    y_coor = circle[1]

    if(y_coor < 0.2 * orig_image.shape[0]):
      red_present = check_red_yellow(orig_image, boundaries, circle)
      
      if(red_present):
        print('Red')
        return
    
    elif(y_coor > 0.2 * orig_image.shape[0] and y_coor < 0.5 * orig_image.shape[0]):
        yellow_present = check_red_yellow(orig_image, boundaries, circle)
       
        if(yellow_present):
          print('Yellow')
          return
    
    else:
      print('Green')
      return

  print('Green')
  

In [ ]:
def generate_circles(image, annotation):

  img = cv2.imread(image, 0)[annotation[1]: annotation[3], annotation[0]: annotation[2]]
  cimg = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
  
 
  circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,1, 0.07 * img.shape[0], param1=30,param2=0.06 * img.shape[0],
                             minRadius= int(0.065*img.shape[0]), maxRadius= int(0.2 *img.shape[0]))
  
  if(circles is None):
    return None
  
  
  circles = np.uint16(np.around(circles))

  circles = circle_filter(img, circles)
 
  for i in circles[0,:]:
    # draw the outer circle
    cv2.circle(cimg,(i[0],i[1]),i[2],(0,255,0),2)
    # draw the center of the circle
    cv2.circle(cimg,(i[0],i[1]),2,(0,0,255),3)
  
  plt.figure()
  plt.imshow(cimg)

  return circles

In [ ]:
def circle_filter(image, circles):
  
  ret_circle = []

  for i in circles[0, :]:
      if(i[1] < 0.7 * image.shape[0] and i[0] > 0.15* image.shape[1] and i[0] < 0.85 * image.shape[1]):
        ret_circle.append(i)

  return np.expand_dims(np.asarray(ret_circle), axis=0)
    


In [ ]:
def check_red_helper(orig_image, boundaries, circle):
    
    y1 = int(circle[1] - circle[2])
    y2 = int(circle[1] + circle[2])
    x1 = int(circle[0] - circle[2])
    x2 = int(circle[0] + circle[2])
    
    temp_image = orig_image[y1:y2, x1:x2]  
    if(temp_image.shape[0] < 1 or temp_image.shape[1] < 1):
      return False
    
    lower, upper = boundaries
	  # create NumPy arrays from the boundaries
    lower = np.array(lower, dtype = "uint8")
    upper = np.array(upper, dtype = "uint8")
	  # find the colors within the specified boundaries and apply
	  # the mask
    mask = cv2.inRange(temp_image, lower, upper)
    area = mask.shape[0] * mask.shape[1]
    count = cv2.countNonZero(mask)
    
    if(count / area > 0.2):
      return True
    return False
  

In [ ]:
def check_red_yellow(orig_image, boundaries, circle):

  ret_val = check_red_helper(orig_image, boundaries, circle)

  if(ret_val):
    return True
  
  return False

In [ ]:
annotation = [450, 581, 796, 1089]
image = 'Test_Imgs/color404_0.png'
main(image, annotation)